In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint
from api_keys import g_key

In [2]:
city_data = pd.read_csv("cities.csv")
city_data

City_ID         City  Cloudiness Country        Date  Humidity    Lat  \
0          0    ostrovnoy           2      RU  1558378754        72  68.05   
1          1    mahebourg          75      MU  1558378503        74 -20.41   
2          2      qaanaaq          25      GL  1558378755        73  77.48   
3          3       zhuhai           0      CN  1558378755        39  40.71   
4          4    cape town          20      ZA  1558378755        76 -33.93   
..       ...          ...         ...     ...         ...       ...    ...   
543      543       yabrud           0      SY  1558378840        32  33.97   
544      544      paraiso           5      MX  1558378898         4  24.01   
545      545      veraval           0      FR  1558378898        62  49.65   
546      546  novyy urgal         100      RU  1558378899        93  51.07   
547      547      tongren           4      CN  1558378899        64  27.72   

        Lng  Max Temp  Wind Speed  
0     39.51     37.50        7.16  
1     57.70     78.80       11.41  
2    -69.36     22.20        2.37  
3    112.04     44.60        4.47  
4     18.42     55.99        8.05  
..      ...       ...         ...  
543   36.66     80.60        8.05  
544 -104.61     84.20       16.11  
545    0.71     61.00        8.05  
546  132.56     49.74        2.68  
547  109.18     55.86        2.84  

[548 rows x 10 columns]

In [3]:
gmaps.configure(api_key=g_key)

locations = city_data[["Lat", "Lng"]].astype(float)
humidity = city_data["Humidity"].astype(float)
fig = gmaps.figure(center = (0,0), zoom_level = 1.5)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Narrow down conditions

#### Ideal vacation: 70-80 degrees

In [4]:
ideal_conditions = pd.DataFrame(city_data.loc[(city_data["Max Temp"] >= 70) & (city_data["Max Temp"]<=80) & (city_data["Wind Speed"]<= 10) & (city_data["Cloudiness"] == 0)])
ideal_conditions.reset_index(inplace = True)
ideal_conditions

index  City_ID          City  Cloudiness Country        Date  Humidity  \
0      7        7         opuwo           0     NaN  1558378523        23   
1     70       70  ponta do sol           0      BR  1558378767        57   
2     88       88      cidreira           0      BR  1558378770        66   
3    138      138     morondava           0      MG  1558378780        69   
4    139      139         vaini           0      IN  1558378780        88   
5    176      176         nador           0      MA  1558378499        40   
6    297      297         mogok           0      MM  1558378844        51   
7    444      444       birjand           0      IR  1558378880        26   

     Lat    Lng  Max Temp  Wind Speed  
0 -18.06  13.84     72.06        5.44  
1 -20.63 -46.00     73.68        5.32  
2 -30.17 -50.22     76.56        5.50  
3 -20.30  44.28     79.08        9.06  
4  15.34  74.49     78.90        1.59  
5  35.17  -2.93     75.20        6.93  
6  22.92  96.51     72.24        2.77  
7  32.86  59.22     71.60        9.17

In [14]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
for index, row in ideal_conditions.iterrows():

    # get restaurant type from df
    location = f"{row['Lat']},{row['Lng']}"

    params = {
        "location": location,
        "radius": 5000,
        "type": "lodging",
        "key": g_key
    }

    response = requests.get(base_url, params=params).json()
    
    results = response['results']
    
    try:
        print(results[0]['name'])
        results[0]['location']
        ideal_conditions.loc[index, 'Nearest Hotel'] = results[0]['name']
#     except (KeyError, IndexError):
#         print("Missing field/result... skipping.")
    ideal_conditions

        
hotel_df = ideal_conditions
hotel_df  

SyntaxError: invalid syntax (3962652042.py, line 24)

In [7]:
marker_locations = hotel_df[['Lat', 'Lng']]
hotel_name = hotel_df["Nearest Hotel"].tolist()
city = hotel_df["City"].tolist()
country = hotel_df["Country"].tolist()

info_box_content = [f"Hotel Name: {hotel_name}, City: {city}, Country: {country}" for hotel_name, city, country in zip(hotel_name, city, country)]

markers = gmaps.marker_layer(marker_locations, info_box_content)
fig.add_layer(markers)
fig

KeyError: 'Nearest Hotel'